# Load the model

In [1]:
# from modelseedpy.fbapkg.reactionusepkg import MinimalMedia

# define the environment path 
from pprint import pprint
from json import dump
import os
local_cobrakbase_path = os.path.join('C:', 'Users', 'Andrew Freiburger','Documents','Argonne','cobrakbase')
os.environ["HOME"] = local_cobrakbase_path

# import the models
import cobrakbase
token = 'CKYZK6AZ5V2CT5ILYP7JAXFPW3OLT6VF'
kbase_api = cobrakbase.KBaseAPI(token)
model = kbase_api.get_from_ws("E_iAH991V2",40576)

# load a community
model1 = kbase_api.get_from_ws("E_iAH991V2",40576)
model2 = kbase_api.get_from_ws("E_iML1515.kb",40576)
com_model = kbase_api.get_from_ws("CMM_iAH991V2_iML1515.kb",40576)
models = [model1, model2]

# prevent excessive warnings
import warnings
warnings.filterwarnings(action='once')

C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


cobrakbase 0.2.8


## Examine the minimal flux media

In [2]:
%run ../modelseedpy/core/minimalmediapkg.py
%time min_flux_media = MinimalMediaPkg.minimize_flux(model, .1)

The minimal flux media consists of 29 compounds and a 0.4074592052031568 total influx, with a growth value of 0.09999999999999967
CPU times: total: 2.2 s
Wall time: 2.19 s


### Defaults to the original objective value

In [7]:
%run ../modelseedpy/core/minimalmediapkg.py
%time min_flux_media = MinimalMediaPkg.minimize_flux(model)

The minimal flux media consists of 51 compounds and a 6097.0693012553975 total influx, with a growth value of 86.91587102429722
CPU times: total: 2.22 s
Wall time: 2.21 s


### contrast with the COBRApy equivalent

In [3]:
def cobra_minimal_flux(model, min_growth):
    from cobra.medium import minimal_medium
    cobra_min_flux_media = minimal_medium(model, min_growth)
    model_copy = model.copy()
    model_copy.medium = cobra_min_flux_media
    print(f"The COBRA minimal flux media consists of {len(cobra_min_flux_media)} compounds and a {sum([flux for rxn, flux in cobra_min_flux_media.items()])} total flux," 
                      f" with a growth value of {model_copy.optimize().objective_value}")
    return cobra_min_flux_media
    
%time cobra_min_flux_media = cobra_minimal_flux(model, .1)

C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\site-packages\cobra\medium\minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()


The COBRA minimal flux media consists of 26 compounds and a 0.40745920520315676 total flux, with a growth value of 0.09999999999999974
CPU times: total: 1.3 s
Wall time: 1.31 s


## Examine the minimal components media

In [ ]:
%run ../modelseedpy/core/minimalmediapkg.py
min_components_media = MinimalMediaPkg.minimize_components(model)
print(f"The minimal components media consists of {len(min_components_media)} compounds and a {sum([ex.flux for ex in min_components_media])} total flux," 
      f" with a growth value of {model.optimize().objective_value}")

In [29]:
from optlang import Variable, Constraint
from modelseedpy import FBAHelper
from optlang.symbolics import Zero
from deepdiff import DeepDiff

def _var_to_ID(var):
    rxnID = var.name
    if "_ru" in rxnID:
        rxnID = rxnID.replace("_ru", "")
    return rxnID

def minimize_components(org_model, minimal_growth=None, printing=True):
    """minimize the quantity of metabolites that are consumed by the model"""
    model = org_model.copy()
    variables = {"ru":{}}
    FBAHelper.add_minimal_objective_cons(
        model, sum([rxn.flux_expression for rxn in model.reactions if "bio" in rxn.id]), minimal_growth)

    # define the binary variable and constraint
    for ex_rxn in FBAHelper.exchange_reactions(model):  # this may need to be relegated to a separate function, depending upon whether additional variables and constraints can be copied with a model 
        # define the variable
        variables["ru"][ex_rxn.id] = Variable(ex_rxn.id+"_ru", lb=0, ub=1, type="binary")
        model.add_cons_vars(variables["ru"][ex_rxn.id])
        # bin_flux: {rxn_bin}*1000 >= {rxn_rev_flux}
        FBAHelper.create_constraint(model, Constraint(Zero, lb=0, ub=None, name=ex_rxn.id+"_bin"),
                                        coef={variables["ru"][ex_rxn.id]: 1000, ex_rxn.reverse_variable: -1})
    print("to objective", f"{len(variables['ru'])} variables are defined")
    FBAHelper.add_objective(model, sum([var for var in variables["ru"].values()]), "min")

    # determine each solution
    solution_dicts = []
    interdependencies = {}
    sol = model.optimize()
    sol_index = 0
    while sol.status == "optimal" and sol_index < 100:
        sol_dict = FBAHelper.solution_to_variables_dict(sol, model)
        solution_dicts.append(sol_dict)
        ## omit the solution from the next search
        FBAHelper.create_constraint(model, Constraint(
            Zero, lb=len(sol_dict)-1, ub=len(sol_dict)-1,name=ex_rxn.id + f"_exclusion_sol{sol_index}"), 
            sol_dict)
        ## search the permutation space by omitting previously investigated solution_dicts
        sol_exchanges = [rxn for rxn in sol_dict if "EX_" in rxn.name]
        interdependencies[sol_index] = _examine_permutations(model, sol_exchanges, variables, sol_dict, sol_index)
        
        ## TODO - intelligently guide future searches with the results of past searches
        
        ## prepare for the new loop
        sol = model.optimize()
        sol_index += 1

def _knockout(org_model, exVar, variables, sol_dict, sol_index):
    # knockout the specified exchange
#     knocked_model = org_model.copy()
    exID = _var_to_ID(exVar)
    coef = {variables["ru"][exID]: 0}
    coef.update({variables["ru"][exVar2.name.replace("_ru", "")]: 1
                 for exVar2 in sol_dict if exVar != exVar2 and "EX_" in exVar2.name})
    FBAHelper.create_constraint(org_model, Constraint(Zero, lb=0.1, ub=None, name=f"{exVar.name}-sol{sol_index}"), coef)
    return org_model.optimize()

def _examine_permutations(model, exchange_ids_to_explore, variables, sol_dict, sol_index):
    for ex in exchange_ids_to_explore:
        sol_dict_sans_ex = sol_dict.copy()
        sol_dict_sans_ex.pop(ex)
        # interdependencies[sol_index][exID] = MinimalMediaPkg._examine_permutations(
        #     exID, sol_dict, sol_index, variables, sol_dict_sans_ex)
        interdependencies = {sol_index:{}}

        ## explore permutations after removing the selected variable
        diff = DeepDiff(sol_dict_sans_ex, FBAHelper.solution_to_dict(
            _knockout(model, ex, variables, sol_dict, sol_index)))
        if diff:  # the addition of new exchanges or altered exchange fluxes are detected after the removed exchange
            for key, changes in diff.items():
                # for change in changes:
                #     print(change)
                changed_reactions = [re.search("(?<=\[\')(.+)(?=\'\])", change).group() for change in changes]
                # this dictionary should be parsed into a list of substitute metabolites and a list of functionally coupled reactions
                for exchange in [rxn for rxn in changed_reactions if "EX_" in rxn]:
                    interdependencies[sol_index][exchange] = _examine_permutations(model, exchange_ids_to_explore, variables, sol_dict, sol_index)
            # coef = {variables["met"][exID]: 0 for cpd in new_mets.keys()}
            # coef.update({variables["met"][exID]: 1 for exID in sol_dict if exID not in new_mets.keys()})
            cpd_name = "_".join(new_mets.keys())
            BaseFBAPkg.build_constraint(self, "met", 0.1, None, coef, f"{cpd_name}-sol{sol_index}")
            new_sol = self.model.optimize()
            new_sol_dict = FBAHelper.solution_to_variables_dict(new_sol, model)
            new_sol_exchanges = [rxn for rxn in sol_dict if "EX_" in rxn.name]
            if new_sol.status != "optimal":
                return interdependencies
            _examine_permutations(model, new_sol_exchanges, variables, new_sol_dict, sol_index)
        return interdependencies

In [30]:
minimize_components(model)

to objective 281 variables are defined


C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\Andrew Freiburger\AppData\Local\Temp\ipykernel_20876\1219899246.py:77: DeprecationWarning: invalid escape sequence \[
  changed_reactions = [re.search("(?<=\[\')(.+)(?=\'\])", change).group() for change in changes]
C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\site-packages\modelseedpy\core\fbahelper.py:137: DeprecationWarning: invalid escape sequence \d
  if re.search('^(cpd\d+)', metabolite.id):


ContainerAlreadyContains: Container '<optlang.container.Container object at 0x000002E0248BB7C0>' already contains an object with name 'EX_cpd00184_e0-sol0'.

## Examine the JENGA minimal media

In [3]:
%run ../modelseedpy/core/minimalmediapkg.py
comm_media, jenga_removals = MinimalMediaPkg.jenga_method(models, com_model, printing=False)

Bacteroides_thetaiotaomicron_VPI-5482.fbamdl.23


C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\site-packages\cobra\medium\minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()


iML1515


C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\site-packages\cobra\medium\minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()


### with compatibilization

In [4]:
%run ../modelseedpy/core/minimalmediapkg.py
comm_media, jenga_removals = MinimalMediaPkg.jenga_method(models, com_model, printing=False, compatibilize=True)

C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\site-packages\cobra\core\model.py:522: DeprecationWarning: add_reaction deprecated. Use add_reactions instead
  warn("add_reaction deprecated. Use add_reactions instead", DeprecationWarning)
modelseedpy.community.mscompatibility - WARNING - ReactionWarning: The metabolite cpd32553_e0 replacement for hspg_degr_1_e0 already exists in the reaction HEPARL1__DASH__e_c0 | 2 cpd00001_e0 + cpd32553_e0 --> 2 cpd00067_e0 + gam26s_e0 + hspg_degr_1_e0 + hspg_degr_2_e0, which inhibits an update.
modelseedpy.community.mscompatibility - WARNING - ModelSEEDError: The old metabolite cpd01399_e0 cross-references ({'AraCyc': ['MALTOTETRAOSE'], 'BiGG': ['maltttr'], 'BrachyCyc': ['MALTOTETRAOSE'], 'KEGG': ['C02052', 'G00457'], 'MetaCyc': ['MALTOTETRAOSE']}) do not overlap with those ({'KEGG': ['C00713', 'G00277', 'G00746'], 'MetaCyc': ['CPD-13556']}) of the new metabolite cpd01399_e0.
modelseedpy.community.mscompatibility - WARNING - IDW



344 reactions were substituted and 13 metabolite IDs were redefined by standardize().
Bacteroides_thetaiotaomicron_VPI-5482.fbamdl.23


ContainerAlreadyContains: Container '<optlang.container.Container object at 0x0000022705E90A30>' already contains an object with name 'bio1'.